In [10]:
from cvzone.HandTrackingModule import HandDetector
import cv2
import cvzone
from keras.models import load_model
import numpy as np
from datetime import datetime, timedelta

USER_OPTIONS_EXPIRY_TIME = 3

GAME_CLASS_ROCK = 0
GAME_CLASS_PAPER = 1
GAME_CLASS_SCISSOR = 2
GAME_CLASS_NONE = 3

GAME_PREDICTION_CONFIDENCE_THRESHOLD = 0.7

def getExpiryTime(second=USER_OPTIONS_EXPIRY_TIME):
    return datetime.now() + timedelta(seconds=second)

def getWinningText(user_option, computer_option):
    if user_option == computer_option:
        return "Tie"
    elif user_option > computer_option:
        if computer_option == GAME_CLASS_ROCK and user_option == GAME_CLASS_SCISSOR:
            return "Computer wins"
        else:
            return "User wins"
    else:
        if user_option == GAME_CLASS_ROCK and computer_option == GAME_CLASS_SCISSOR:
            return "User wins"
        else:
            return "Computer wins"

def startGame():
    model = load_model('./model/keras_model.h5', compile=False)
    
    cap = cv2.VideoCapture(0)
    detector = HandDetector(detectionCon=0.8, maxHands=1)
    
    font = cv2.FONT_HERSHEY_SIMPLEX

    with open('./model/labels.txt') as f:
        lines = f.readlines()

    labels = []
    for line in lines:
        labels.append(line.split(" ")[1].replace("\n", ""))

    computer_class = np.random.randint(0, 3)
    computer_label = labels[computer_class]

    prev_choice = GAME_CLASS_NONE # initial choice = None
    expiry_time = None
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    is_playing = True
    winning_text = ""

    while True:
        ret, frame = cap.read()

        # if it is not playing the game, show instruction and and the frame catured from webcam only
        if is_playing == False:
            cv2.putText(frame,
                        winning_text, 
                        (20, 50), 
                        font, 1, 
                        (255, 255, 255), 
                        2, 
                        cv2.LINE_4)

            cv2.putText(frame,
                    f'Press r to restart, press q to quit the game', 
                    (20, 100), 
                    font, 1, 
                    (255, 255, 255), 
                    2, 
                    cv2.LINE_4)

            cv2.imshow('frame', frame)
            key = cv2.waitKey(1) & 0xFF
            
            # Press q to close the window
            if key == ord("q"):
                break

            # Press r to restart the game
            if key == ord("r"):
                is_playing = True
                computer_class = np.random.randint(GAME_CLASS_ROCK, GAME_CLASS_SCISSOR+1)
                computer_label = labels[computer_class]

                prev_choice = GAME_CLASS_NONE # initial choice = None
                expiry_time = None
                
        else:
            # if the game is running, try to find a hand from the web cam input, predict the output only using the hand
            
            # top left instruction
            cv2.putText(frame,
                f'Hold your options for {USER_OPTIONS_EXPIRY_TIME} seconds', 
                (20, 50), 
                font, 1, 
                (255, 255, 255), 
                2, 
                cv2.LINE_4)

            cv2.putText(frame,
                    f'Press q to quit the game', 
                    (20, 100), 
                    font, 1, 
                    (255, 255, 255), 
                    2, 
                    cv2.LINE_4)

            # find hands, img will be and array of hand in dictionary with key "lmlist", "bbox", "center", where bbox is the
            # position of the hand
            img = detector.findHands(frame, draw=False)

            # only take care the case with one hand detected
            if img and len(img)==1:
                bbox = img[0]["bbox"]
                print(f"hand located [{bbox}]")
                # print(f"frame {frame}")

                # try capture a little bit more to ensure the whole hand is covered in the frame
                startX = max(bbox[0]-80, 0)
                endX = min(bbox[0] + bbox[2] + 80, frame.shape[1])

                startY = max(bbox[1]-80, 0)
                endY = min(bbox[1] + bbox[3] + 80, frame.shape[0])

                # use array slicing to select position with the hand
                frame_hand = frame[startY:endY, startX:endX, :]
                print(f"Capture image at [{startX}, {startY}, {endX}, {endY}]")

                
                
                if frame_hand.shape[0] > 0 and frame_hand.shape[1] > 0:
                    # draw a bounding box in the area of the hand
                    cvzone.cornerRect(frame, (startX, startY, endX-startX, endY-startY))

                    # cv2.imshow('frame', frame_hand)

                    # resize and normalize the image
                    resized_frame = cv2.resize(frame_hand, (224, 224), interpolation = cv2.INTER_AREA)
                    image_np = np.array(resized_frame)
                    normalized_image = (image_np.astype(np.float32) / 127.0) - 1 # Normalize the image
                    data[0] = normalized_image

                    # predict the class with the model
                    prediction = model.predict(data)

                    # print(prediction)

                    predict_class = np.argmax(np.squeeze(prediction))
                    predict_prob = np.squeeze(prediction)[predict_class]

                    # if the predicte probability < than the condifence threshold, 
                    # reject the predicton and we can assume no class has been matched
                    if predict_prob < GAME_PREDICTION_CONFIDENCE_THRESHOLD:
                        predict_class = GAME_CLASS_NONE

                    predict_label = labels[predict_class]
                    print(predict_label)

                    if predict_class < GAME_CLASS_NONE:
                        
                        # check if user keep their option, if not, record the new option and restart the timer
                        if prev_choice != predict_class:
                            expiry_time = getExpiryTime()

                            print(f"User select new option {predict_label}, update expiry time {expiry_time}")

                        # user hold his/her previous option, check if the user has hold it for a while, 
                        # if yes, we can decide who is the winner
                        elif datetime.now() > expiry_time:
                            winning_text = getWinningText(predict_class, computer_class)
                            winning_text = f'Computer option is {computer_label}, your option is {predict_label}, {winning_text}'

                            is_playing = False

                    prev_choice = predict_class
                
                # hand is found, but we can't capture it from the frame
                else:
                    prev_choice = GAME_CLASS_NONE

                cv2.imshow('frame', frame)

            # no hand is found
            else:
                prev_choice = GAME_CLASS_NONE
                cv2.imshow('frame', frame)

            # just leave the option to quit the game
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break


    # After the loop release the cap object
    cap.release()
    # Destroy all the windows
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

startGame()

hand located [(379, 403, 96, 164)]
Capture image at [299, 323, 555, 647]
None
hand located [(267, 328, 156, 263)]
Capture image at [187, 248, 503, 671]
None
hand located [(269, 331, 155, 263)]
Capture image at [189, 251, 504, 674]
None
hand located [(270, 331, 155, 265)]
Capture image at [190, 251, 505, 676]
None
hand located [(269, 331, 156, 272)]
Capture image at [189, 251, 505, 683]
None
hand located [(267, 330, 154, 272)]
Capture image at [187, 250, 501, 682]
None
hand located [(263, 329, 158, 276)]
Capture image at [183, 249, 501, 685]
None
hand located [(262, 329, 157, 273)]
Capture image at [182, 249, 499, 682]
None
hand located [(262, 331, 158, 269)]
Capture image at [182, 251, 500, 680]
None
hand located [(260, 331, 159, 272)]
Capture image at [180, 251, 499, 683]
None
hand located [(262, 329, 158, 273)]
Capture image at [182, 249, 500, 682]
None
hand located [(262, 330, 157, 274)]
Capture image at [182, 250, 499, 684]
None
hand located [(264, 331, 156, 274)]
Capture image at 

In [ ]:
from cvzone.HandTrackingModule import HandDetector
import cv2

cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8, maxHands=2)
while True:
    # Get image frame
    success, img = cap.read()
    # Find the hand and its landmarks
    hands, img = detector.findHands(img)  # with draw
    # hands = detector.findHands(img, draw=False)  # without draw

    if hands:
        # Hand 1
        hand1 = hands[0]
        lmList1 = hand1["lmList"]  # List of 21 Landmark points
        bbox1 = hand1["bbox"]  # Bounding box info x,y,w,h
        centerPoint1 = hand1['center']  # center of the hand cx,cy
        handType1 = hand1["type"]  # Handtype Left or Right

        fingers1 = detector.fingersUp(hand1)

        if len(hands) == 2:
            # Hand 2
            hand2 = hands[1]
            lmList2 = hand2["lmList"]  # List of 21 Landmark points
            bbox2 = hand2["bbox"]  # Bounding box info x,y,w,h
            centerPoint2 = hand2['center']  # center of the hand cx,cy
            handType2 = hand2["type"]  # Hand Type "Left" or "Right"

            fingers2 = detector.fingersUp(hand2)

            # Find Distance between two Landmarks. Could be same hand or different hands
            length, info, img = detector.findDistance(lmList1[8], lmList2[8], img)  # with draw
            # length, info = detector.findDistance(lmList1[8], lmList2[8])  # with draw
    # Display
    cv2.imshow("Image", img)
    cv2.waitKey(1)
cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


KeyboardInterrupt: 